In [2]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt

train = pd.read_csv('TRAIN_nonMissing_withTime_PCA.csv')
test = pd.read_csv("TEST_nonMissing_withTime_PCA.csv")

feature_names = train.columns # first is just original index
features_removed = ['Unnamed: 0'] # original index before ordering
feature_names = feature_names[~feature_names.isin(features_removed)]
train = train[feature_names]
test = test[feature_names]
train['DATE'] = pd.to_datetime(train['DATE'])
test['DATE'] = pd.to_datetime(test['DATE'])
train = train.sort(columns = ['ID','DATE','SLOTID'])
test = test.sort(columns = ['ID','DATE','SLOTID'])


In [36]:
#add some more features:
#isWeekday, isHoday
import datetime
import math
holidays = pd.read_csv("holiday2016.csv")
year = 2016
Allholidays=[]
for i in range(len(holidays['month'])):
    month = holidays.loc[i,'month']
    day = holidays.loc[i,'day']
    Allholidays.append(datetime.date(year,month,day))
    
def moreFeatures(train):
    dates = train['DATE']
    day =[]
    isWeekend =[]
    isHoliday=[]
    for dt in dates:
        date = dt.date()
        day.append(date.strftime("%A"))
        if date.strftime("%A") =="Saturday" or date.strftime("%A") =="Sunday":
            isWeekend.append(1)
        else:
            isWeekend.append(0)
            
        isHoliday.append(date in Allholidays)
    train.loc[:,'weekday'] = pd.Series(day, index=train.index) 
    train.loc[:,'isWeekend'] = pd.Series(isWeekend, index=train.index) 
    train.loc[:,'isHoliday'] = pd.Series(isHoliday, index=train.index) 
    train.loc[:,'DayPeriod_COS1'] = pd.Series([math.cos(sid*1.0/144) for sid in train['SLOTID']],index=train.index)
    train.loc[:,'DayPeriod_COS2'] = pd.Series([math.cos(sid*2.0/144) for sid in train['SLOTID']],index=train.index)
    train.loc[:,'DayPeriod_SIN1'] = pd.Series([math.sin(sid*1.0/144) for sid in train['SLOTID']],index=train.index)
    train.loc[:,'DayPeriod_SIN2'] = pd.Series([math.sin(sid*2.0/144) for sid in train['SLOTID']],index=train.index)

moreFeatures(train)
moreFeatures(test)
#train.to_csv("TRAIN_with_AllFeatures.csv")       
#test.to_csv("TEST_with_AllFeatures.csv")    

def correctDataType(train):
    categorial_features = ['ID','WEATHER_1','WEATHER_2','WEATHER_3','weekday']
    #train[categorial_features] = train[categorial_features].apply(pd.Categorical)
    train['ID'] =train['ID'].astype("category")
    train['WEATHER_1'] =train['WEATHER_1'].astype("category")
    train['WEATHER_2'] =train['WEATHER_2'].astype("category")
    train['WEATHER_3'] =train['WEATHER_3'].astype("category")
    train['weekday'] =train['weekday'].astype("category", )
    #train.loc[:,train.columns[~train.columns.isin(categorial_features)]] = \
    #     train[train.columns[~train.columns.isin(categorial_features)]].astype(float)
    
correctDataType(train)  
correctDataType(test)


In [37]:
from math import fabs 
def Accuracy (TRAIN, ypred):
    PRED = pd.DataFrame({'ID':TRAIN['ID'].tolist(),'GAP':[val for val in ypred]})
    IDs = TRAIN['ID'].drop_duplicates()
    s=0
    for id in IDs:
        t = 0
        GAP = TRAIN[TRAIN['ID'].isin([id])]['GAP'].tolist()
        GAP_pred = PRED[PRED['ID'].isin([id])]['GAP'].tolist()
        count = 0 
        for i in range(len(GAP)):
            if GAP[i] > 0:
                t = t + fabs( (GAP[i]-GAP_pred[i])*1.0/GAP[i])
                count = count+1
        s = s +   1.0 *t / count   
    return s/len(IDs) 


def Accuracy2 (GAP, ypred):
    s = 0;
    for i in range(len(ypred)):
        if GAP[i] > 0:
            s = s + fabs( (GAP[i]-ypred[i])*1.0/GAP[i])  
    return s/len(ypred) 

#Accuracy(train2, ypred_rf2)

In [209]:
#divide the data into train and validiation, and test set
nTRAIN = len(train['ID'])
sample_validation = range(nTRAIN-66*144*2, nTRAIN)
sample_train = range(nTRAIN-66*144*2)

train1 = train.iloc[sample_train,]
train2 = train.iloc[sample_validation]
slotids_test = test['SLOTID'].drop_duplicates()
train2 = train2[train2['SLOTID'].isin(slotids_test)]

#pd.Series(range(len(train['GAP'])))[~pd.Series(range(len(train['GAP']))).isin(sample_validation)]
GAP_tr = train['GAP']
GAP_tr1 = train1['GAP']
GAP_tr2 = train2['GAP']
GAP_te = test['GAP']

feature_names = train.columns[:]
feature_removed = ['DATE','GAP','weekday']
feature_names = feature_names[~ feature_names.isin(feature_removed)] # remove some IDs
features_tr1 = train1[feature_names]
features_tr2 = train2[feature_names]
features_tr = train[feature_names]
features_te = test[feature_names]
 

from math import log1p, expm1
from sklearn import tree
DTRegressor = tree.DecisionTreeRegressor(max_depth=None,  min_samples_split=5, 
                             random_state=0)
X = features_tr1
y = GAP_tr1
reg = DTRegressor.fit(X, y)
ypred_tree1 = reg.predict(features_tr2)
ypred_tree1 = np.round(ypred_tree1)
print Accuracy(train2,ypred_tree1)


X = features_tr1
y = [log1p(val) for val in GAP_tr1.tolist()]
reg = DTRegressor.fit(X, y)
ypred_tree2 = reg.predict(features_tr2)
ypred_tree2 = np.round([expm1(val) for val in ypred_tree2])
print Accuracy(train2,ypred_tree2)



from sklearn.cross_validation import cross_val_score
scores = cross_val_score(DTRegressor, features_tr, GAP_tr) # what type of error 
print scores



from math import log1p, expm1
from sklearn.ensemble import RandomForestRegressor
rfRegressor = RandomForestRegressor(n_estimators=150, max_features=10, min_samples_split=5)
X = features_tr1
y =  GAP_tr1
reg = rfRegressor.fit(X, y)
ypred_rf1 = reg.predict(features_tr2)
ypred_rf1 = np.round(ypred_rf1)

print Accuracy(train2,ypred_rf1)

#take log first, minimize MSE and then scale back
rfRegressor = RandomForestRegressor(n_estimators=150, max_features=10, min_samples_split=5)
X = features_tr1
y = [log1p(val) for val in GAP_tr1.tolist()]
reg = rfRegressor.fit(X, y)
ypred_rf = reg.predict(features_tr2)
ypred_rf2 = np.round([expm1(val) for val in ypred_rf])
print Accuracy(train2,ypred_rf2)


1.10523510957
1.04648580465
[ 0.6966433   0.70593566  0.75474322]
0.415573536809
0.464293947599


In [ ]:
1.10523510957
1.04648580465
[ 0.69445639  0.70651343  0.76441657]
0.42266738693   #100 tree
0.488925819351

In [ ]:
#divide the data into train and validiation, and test set

def forecastError(train, test, k):
    nTRAIN = len(train['ID'])
    feature_names = train.columns
    sample_validation = range(nTRAIN-66*144*k, nTRAIN)
    sample_train = range(nTRAIN-66*144*k)

    train1 = train.iloc[sample_train,]
    train2 = train.iloc[sample_validation]
    slotids_test = test['SLOTID'].drop_duplicates()
    train2 = train2[train2['SLOTID'].isin(slotids_test)]

    #pd.Series(range(len(train['GAP'])))[~pd.Series(range(len(train['GAP']))).isin(sample_validation)]
    GAP_tr = train['GAP']
    GAP_tr1 = train1['GAP']
    GAP_tr2 = train2['GAP']
    GAP_te = test['GAP']

    feature_names = train.columns[:]
    feature_removed = [DATE','GAP','weekday']
    feature_names = feature_names[~ feature_names.isin(feature_removed)] # remove some IDs
    features_tr1 = train1[feature_names]
    features_tr2 = train2[feature_names]
    features_tr = train[feature_names]
    features_te = test[feature_names]

    from math import log1p, expm1
    from sklearn.ensemble import RandomForestRegressor
    rfRegressor = RandomForestRegressor(n_estimators=100, max_features=10, min_samples_split=5)
    X=features_tr1
    y =  GAP_tr1
    rfg = rfRegressor.fit(X, y)
    ypred_rf1 = rfg.predict(features_tr2)
    ypred_rf1 = np.round(ypred_rf1)

    #take log first, minimize MSE and then scale back
    y = [log1p(val) for val in GAP_tr1.tolist()]
    rfg = rfRegressor.fit(X, y)
    ypred_rf2 = rfg.predict(features_tr2)
    ypred_rf2 = np.round([expm1(val) for val in ypred_rf2])

    return Accuracy(train2,ypred_rf1),Accuracy(train2,ypred_rf2)

print forecastError(train,test,1)


In [212]:

##rf prediction without log tranformation
from sklearn.ensemble import RandomForestRegressor
tfRegressor = RandomForestRegressor(n_estimators=200, max_features=10, min_samples_split=5)
reg = tfRegressor.fit(features_tr, GAP_tr)
ypred_rf = reg.predict(features_te)
ypred_rf =np.round(ypred_rf)

def submitResult(ypred):
    # create the first guess prediction: with the previous gap
    import csv
    test = pd.read_csv("TEST_nonMissing_withTime.csv")
    import datetime
    test['DATE'] = pd.to_datetime(test['DATE'])
    timeID = []
    distID = []
    #test1 = test[['ID','DATE','SLOTID','GAP']]
    for i in range(len(test['ID'])):
        row = test.iloc[i,:]
        distID.append(row['ID'])
        date = row['DATE'].date()
        if date.month <10:
            timeID.append(str(date.year)+ '-' +'0'+str(date.month) + '-' +str(date.day) + '-' +str(row['SLOTID']+1)) # 
        else:
            timeID.append(str(date.year)+ '-' +str(date.month) + '-' +str(date.day) + '-' +str(row['SLOTID']+1)) # 
      
        #pred.append(row['GAP_1'])
    submitResult =pd.DataFrame({'distID': distID,
                            'timeID': timeID,
                            'GAP': ypred})   # predicted value
    submitResult = submitResult[['distID','timeID','GAP']]
    timeSLOTIDs = pd.read_csv('./test_set_1/read_me_1.txt')
    submitResult = submitResult[submitResult2['timeID'].isin(timeSLOTIDs ['需预测时间片:'].tolist())]
    return submitResult

submitResult2 = submitResult(ypred_rf)


SyntaxError: invalid syntax (<ipython-input-212-6c45ba6582e5>, line 35)

In [223]:
print submitResult2[:10]
print len(submitResult2['GAP'])
submitResult2.to_csv("submitResult3.csv", index = None, header = None)


   distID          timeID  GAP
0       1   2016-01-22-46   10
1       1   2016-01-22-58    8
2       1   2016-01-22-70    5
3       1   2016-01-22-82    5
4       1   2016-01-22-94    5
5       1  2016-01-22-106   15
6       1  2016-01-22-118    7
7       1  2016-01-22-130    8
8       1  2016-01-22-142    9
9       1   2016-01-24-46    5
